# Digital Identity

Digital identity is everything about you that exists online, such as your name, email, photos, login details, posts, and even how you behave on the internet.  
It’s your online self, like the version of you that websites, apps, and algorithms recognise, track, and use to give access, show ads, or personalise what you see.

| Layer | Desc | Examples |
|-------|------|----------|
| Identifiers | Unique attributes tat identify you | Username, email, device ID, IP address, branding |
| Credentials | Data that verifies authenticity | Passwords, biometrics (fingerprint, face scan), tokens, digital signatures |
| Behavioral Data | How you act online | Search patterns, click behavior, language use, social media habits |
| Transactional Data | Records of your actions | Purchase history, login records, blockchain transactions |
| Derived or Inferred Attributes | AI-generated characteristics | Psychographic profile, sentiment, interests, political leaning |

#### Types of Digital Identity
1. Centralized identity:
    - a digital identity model in which one central authority or organization controls the storage, verification, and management of user identities.
    - Managed by a single organization (e.g., Google, Facebook, government portals).
    - Users depend on the provider to store and secure their data.
    - User → Identity Provider (IDP/verify username,password,keys) → Service Provider (SP/service App google)
    - E.g: Login to google (ISP) to using YT (SP)
2. Federated Identity:
    - Allow users to use one identity across multiple platforms (OAuth/SAML)
    - E.g: Register in Academic platform using google (OAuth)
3. Self-Sovereign Identity (SSI):
    - A decentralized model using blockchain or cryptography, giving full control to the user.
    - User stores and shares only selected credentials.

#### Digital identity in practice
| Where | Example |
|-------|---------|
| Government | eKYC, Digital Passports, Biometric ID |
| Finance | Online banking, Digital wallets, KYC/AML |
| Social Media | Profile based digital personas reflect behavior/Influence |
| Enterprise | RBAC, SSO |
| Blockchain/Web3 | Wallet address as identity (pseudonymous but traceable). |

#### Risk & Challange
| Risk | Why |
|------|-----|
| Privacy erosion | Data is often collected without explicit consent. |
| Data Fragmentation | Different identities exist on different platforms with no unified control. |
| Identity Theft & Fraud | Hacking or phishing attacks can exploit credentials. |
| Bias in Algorithmic Identity | |

#### Example flow with Lasswell Model
| Element | classic | Digital identity |
|---------|---------|-------------|
| Who/Sender | Human Communicator (Journalist, speaker) | Digital identity (username, bio, photo), Behavioral data (like,searches,location, time online) |
| Says What/Message | Crafted message (speech,article,image) | The content intentionally share (posts, videos, comments)/unconscious data generate (clicks, scrolls, patterns) |
| In which channel | TV, Radio, newspaper etc | Social media, search engines, apps, online communicative |
| To whom (Audience) | Passive receiver | Human audience (friends, followers, employers, strangers)/Algorithmic audience (system that reads, profiles, categorizes your behavior) |
| With what effect | Persuasion, knowledge, influence | Personal branding, social influence, Feedback loops |


#### Example Code Logic

In [9]:
import praw as pw
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import pipeline
from datetime import datetime, timezone
 
# sender -> message -> channel -> receiver -> effect

secret = "vyvT97jyz0V-lgBRVnlC0WQjM7AwzA"
clientID = "QDwXqDL-Oal4Ho-ATuxx1Q"
redditUA = "testscript by u/Kooky_Boysenberry943"
reddit = pw.Reddit(client_id=clientID, client_secret=secret, user_agent=redditUA)
user = reddit.redditor("kawaiicelyynna")

profiles = []
posts_data = []

def fetchSender(user):
    sub = user.subreddit
    
    sender = {
        "username": user.name,
        "ID": user.id,
        "title": sub.title,
        "bio": sub.public_description,
        "follower": sub.subscribers,
        "link_karma": user.link_karma,
        "comment_karma": user.comment_karma,
        "acc_created_time": datetime.fromtimestamp(sub.created_utc).strftime("%Y-%m-%d %H:%M:%S"),
        "is_moderator": user.is_mod,
        "has_verified": user.has_verified_email,
        "trophies": [trophy.name for trophy in user.trophies()]   
    }
    
    return sender

def convertURL(username, postID, title):
    url = f"https://www.reddit.com/user/{username}/comments/{postID}/{title.lower().replace(" ","_")}"

    return url
    
def fetchMessage(user):
    latest_post = next(user.submissions.new(limit=2), None)
    if latest_post:
        message = {
            "title": latest_post.title,
            "post_url": latest_post.url,
            "post_ID": latest_post.id,
            "comments": latest_post.num_comments,
            "post_desc": latest_post.selftext,
            "created": datetime.fromtimestamp(latest_post.created_utc).strftime("%Y-%m-%d %H:%M:%S")
        }
        return message
    else:
        return f"{user.name} has no recent posts."


msg = fetchMessage(user)

def audience(msg, username):
    url = convertURL(username, msg["post_ID"], msg["title"])
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)

    comments_data = [
        {
            "author": str(comment.author),
            "score": comment.score,
            "created_utc": comment.created_utc,
            "retweet": submission.num_crossposts,
            "comments": comment.body
        } for comment in submission.comments.list()
    ]
    return comments_data

audience = audience(msg, user.name)

def effect(text):
    # vader
    analyzer = SentimentIntensityAnalyzer()
    vd_result = analyzer.polarity_scores(text)

    # textblob
    tb_result = TextBlob(text).sentiment.polarity

    # transformer
    sentiment = pipeline("sentiment-analysis")
    tf_result = sentiment(text)
    
    return list((vd_result,tb_result,tf_result))

def weigh_avr(senti):
    vd = senti[0]
    tb = senti[1]
    tf = senti[2][0]

    threshold = 0.05

    # vader
    if vd["compound"] >= threshold:
        vd_sentiment = 1
    elif vd["compound"] <= -threshold:
        vd_sentiment = -1

    else:
        vd_sentiment = 0 # Neutral

    # textblob
    if tb >= threshold:
        tb_sentiment = 1
    elif tb <= -threshold:
        tb_sentiment = -1
    else:
        tb_sentiment = 0

    # transformers
    if tf["label"] == "POSITIVE" and tf["score"] >= threshold:
        tf_sentiment = 1
    elif tf["label"] == "NEGATIVE" and -tf["score"] <= threshold:
        tf_sentiment = -1
    else:
        tf_sentiment = 0


    cap_threshold = 0.95

    # cap val
    vd_cap = max(min(vd_sentiment,cap_threshold),-cap_threshold)
    tb_cap = max(min(tb_sentiment,cap_threshold),-cap_threshold)
    tf_cap = max(min(tf_sentiment,cap_threshold),-cap_threshold)

    weighted_avr = (0.5*tf_cap + 0.3*vd_cap + 0.2*tb_cap)

    return weighted_avr

sentiment_avr = sum([weigh_avr(effect(text["comments"])) for text in audience]) / len(audience)

lasswell = {
    'sender': fetchSender(user),
    'message': msg,
    'channel':'reddit',
    'audience': audience,
    'effect': {'sentiment_avr':sentiment_avr}
}

lasswell

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilb

{'sender': {'username': 'kawaiicelyynna',
  'ID': 'nobrc64i4',
  'title': 'heyitscee',
  'bio': '',
  'follower': 0,
  'link_karma': 120,
  'comment_karma': 60,
  'acc_created_time': '2023-11-11 21:59:05',
  'is_moderator': True,
  'has_verified': True,
  'trophies': ['One-Year Club']},
 'message': {'title': 'Another client feedback',
  'post_url': 'https://i.redd.it/nk4dl15omm8f1.jpeg',
  'post_ID': '1lia3p6',
  'comments': 6,
  'post_desc': 'Thank you so much. \nI really appreciate po yung mga ganitong feedback mas lalo ka gaganahan magbigay ng good quality service sa mga client mo 🙂 ',
  'created': '2025-06-23 15:09:41'},
 'channel': 'reddit',
 'audience': [{'author': 'Consistent-Name1726',
   'score': 2,
   'created_utc': 1752325932.0,
   'retweet': 0,
   'comments': 'I had the chance to work with her on my papers, and she was incredibly helpful throughout the process. She took initiative in solving issues I didn’t even notice at first and went above and beyond to ensure everything